# Minimum Edit Distance

Minimum Edit Distance is one of the simplest and most efficient way to measure how similar two strings are. One useful usage of Minimum Edit Distance is spelling correction. The minimum edit distance is defined as the minimum number of editing operations needed to transform one string into another. 

There is a simple python library called levenshtein that calculates the Levenshtein distance. This is the minimum edit distance with the simplest weighting factor in which each of the three possible operation (insert, substitute, delete) gets a weight of 1.

We can think of finding the MED as a search task, in which we are searching for the shortest path - a seduence of edits - from one string to another. The space of all possible edits is enormous, so we can't serach naively. <b>Dynamic Programming</b> might be a good solution to do that.

In [1]:
def find_distance (first_substring, second_substring):
    distance = 0;
    chars = [];
    for f_char in first_substring:
        if f_char not in second_substring:
            distance += 1;
        else:
            chars.append(f_char);
    for s_char in second_substring:
        if s_char not in first_substring:
            distance += 1;
        else:
            try:
                chars.remove(s_char);
            except:
                distance = distance + 1;
    return distance + len(chars);

In [2]:
first_string = input("Please enter the first string ");
second_string = input("Please enter the second string ");

Please enter the first string tom
Please enter the second string dsf


In [ ]:
first_length = len(first_string);
second_length = len(second_string);
calc_dis = [[0 for x in range(second_length+1)] for y in range(first_length+1)];

In [ ]:
input_string = "";
output_string = "";

In [ ]:
for i in range(1,first_length+1):
    calc_dis [i][0] = calc_dis[i-1][0]+1;
for j in range(1,first_length+1):
    calc_dis [0][j] = calc_dis[0][j-1]+1;
display(calc_dis)

In [135]:
for i in range(1,first_length+1):
    for j in range(1,second_length+1):
        calc_dis [i][j] = find_distance(first_string[:i], second_string[:j]);

In [136]:
display(calc_dis);

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [1, 2, 3, 4, 5, 6, 7, 6, 7, 8],
 [2, 3, 4, 5, 6, 7, 8, 7, 8, 7],
 [3, 4, 5, 6, 7, 8, 7, 6, 7, 6],
 [4, 3, 4, 5, 6, 7, 6, 5, 6, 5],
 [5, 4, 5, 6, 7, 8, 7, 6, 7, 6],
 [6, 5, 6, 7, 8, 9, 8, 7, 8, 7],
 [7, 6, 7, 8, 9, 10, 9, 8, 9, 8],
 [8, 7, 8, 9, 10, 11, 10, 9, 8, 7],
 [9, 8, 9, 10, 11, 12, 11, 10, 9, 8]]

In [143]:
print("The Levenshtein Distance between "+first_string+" and "+second_string+" is: "+str(calc_dis[second_length][first_length]))

The Levenshtein Distance between intention and execution is: 8


## Naive Spelling Correction

Now we might be able to use our code for some simple spelling correction. If we'd have a dictionary of words from multiple Wikipedia entries, we might be able to identify naively the word that our user intended to write. 

In [2]:
import wikipedia
import re
import random

In [3]:
def get_page_contents(number_of_wiki_pages):
  page_contents = [];
  while len(page_contents) < number_of_wiki_pages:
    page_title = wikipedia.random(pages = 1);
    try:  
      text_content = wikipedia.WikipediaPage(title=page_title).content;
    except wikipedia.exceptions.DisambiguationError as e:
      try:
        page_title = e.options[0];
        text_content = wikipedia.WikipediaPage(title=page_title).content;
      except wikipedia.exceptions.WikipediaException as e:
        continue;
    print("Page Number "+str(len(page_contents) + 1)+" is: "+page_title);
    page_contents.append(text_content);
  return page_contents;

In [4]:
def get_words_from_content(page_contents):
  word_tokens = [];
  for content in page_contents:
      words = content.split(" ");  
      for word in words:
        new_word = ""
        for char in re.findall("[a-zA-Z]", word):
          if char.isupper():
            new_word = new_word + char.lower();
          else:
            new_word = new_word + char; 
        if new_word not in word_tokens and len(new_word) > 0:
          word_tokens.append(new_word);
  return word_tokens;

In [6]:
number_of_wiki_pages = 100;
page_contents = get_page_contents(number_of_wiki_pages);
word_tokens = get_words_from_content(page_contents);

Page Number 1 is: Nikosthenes
Page Number 2 is: Paul Blackwell
Page Number 3 is: Adegramotide
Page Number 4 is: Windom, Kansas
Page Number 5 is: Sülze (Bergen)
Page Number 6 is: Junior Eurovision Song Contest 2012
Page Number 7 is: Welsh calendar
Page Number 8 is: 1964 Hama riot
Page Number 9 is: Caroline Green
Page Number 10 is: Kirchheim (Neckar) station
Page Number 11 is: Augnablik Kópavogur
Page Number 12 is: José Aponte Hernández
Page Number 13 is: Emigsville, Pennsylvania
Page Number 14 is: Barren Grounds Nature Reserve
Page Number 15 is: There There (novel)
Page Number 16 is: Bazilio Olara-Okello
Page Number 17 is: Jeeves and the Impending Doom
Page Number 18 is: Chris Smith (pitcher, born 1981)
Page Number 19 is: Lee Min-ho (actor)
Page Number 20 is: Munster Schools Rugby Senior Cup
Page Number 21 is: 60 Serpentis
Page Number 22 is: HaDugmaniyot (season 2)
Page Number 23 is: Juan Bautista Garcia
Page Number 24 is: Goudi
Page Number 25 is: Perthes-lès-Brienne
Page Number 26 is: 

In [29]:
word_from_user = input("Please search for a word ");
distances = {};
for word in word_tokens:
    distances[word] = find_distance(word_from_user,word);

Please search for a word monkey


In [30]:
print("Your top suggestions for "+word_from_user+" are: ")
index = 0;
for w in sorted(distances, key=distances.get, reverse=False):
    if index <= 5:
        print(w);
        index += 1;

Your top suggestions for monkey are: 
money
koreanmy
monk
one
economy
enemy


## Improvements

Results are looking mediocre at the moment. We might use some heuristics to improve results.

In [16]:
def add_heuristics(word_one, word_two):
    addition = 0;
    
    for i in range(1,min(len(word_one),len(word_two))):
        if word_one[i] != word_two[i]:
            addition += 1;
    
    return addition    

In [31]:
word_from_user = input("Please search for a word ");
distances = {};
for word in word_tokens:
    distances[word] = find_distance(word_from_user,word) + add_heuristics(word_from_user,word);

Please search for a word monkey


In [32]:
print("Your top suggestions for "+word_from_user+" are: ")
index = 0;
for w in sorted(distances, key=distances.get, reverse=False):
    if index <= 5:
        print(w);
        index += 1;

Your top suggestions for monkey are: 
monk
money
mon
no
ken
moe
